Author: Ming Huang

- Last updated: 05/25/2016
- By: Ming Huang

# Objectives

- Use Psycopg2 to connect to Postgres from Python
- Explain how to run SQL queries from Python
- Explain how to create dynamic SQL queries through Python string formatting

# Why?

* Leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
* Very useful for scaled data pipelines, pre-cleaning, data exploration
* Allows for dynamic query generation and hence automations

# Psycopg2

A Python library that allows for connections to an existing Postgres database to execute queries and retrieve data.

### Documentation (Includes Installation Instruction)

- http://initd.org/psycopg/docs/install.html

# Walkthrough 1: Creating a database from Python

#### Step 1: Import psycopg2 into our namespace

In [ ]:
import psycopg2 as pg2

#### Step 2: Create a connection with Postgres

This is equivalent to logging into the Postgres shell (using psql in terminal)

If you need to specify the database server url,  you can do it using the host parameter.

In [ ]:
conn = pg2.connect(dbname='postgres', user='minghuang')

#### Step 3: Set autocommit to True

It turns out that database changes are not commited (saved) until you tell Psycopg2 to commit some actions.  So until you commit, all your changes are only done in memory.  This is a big problem when it comes to creating databases, because Postgres does not allow the creation of database in just memory. So we need to make sure everything we do gets automatically commited.

In [ ]:
conn.autocommit = True

#### Step 4: Create the Cursor

Psycopg2 interacts with Postgres through the means of cursors. Cursors are control structures that allows Psycopg2 to execute queries and traverse through query results (if there's any).

In [ ]:
cur = conn.cursor()

#### Step 5: Create a database

With our cursor, we can now execute SQL commands.

In [ ]:
cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')

#### Step 6: Disconnect from the cursor and database

After you're done, don't forget to close the cursor or connection.  Otherwise Postgres will complain because it thinks someone else is using it.

In [ ]:
cur.close() # This is optional
conn.close()

# Walkthrough 2: Lets use our new database

#### Step 1: Connect to our database

Now that we have a created database, we can connect to it.  Instead of connect to 'postgres' which takes us into the shell, we can just connect to the database.  This is equivalent to using 'psql temp' in the terminal.

In [ ]:
conn = pg2.connect(dbname='temp', user='minghuang')

#### Step 2: Create a cursor

In [ ]:
cur = conn.cursor()

#### Step 3: Create a new table

Since we're working from inside Python, we can store our query inside strings and just reference them later.

In [ ]:
query = '''
        CREATE TABLE logins (
            userid integer
            , tmstmp timestamp
            , type varchar(10)
        );
        '''
cur.execute(query)

#### Step 4: Insert .csv data into new table

Lets insert some data so we can play with them.

In [ ]:
query = '''
        COPY logins 
        FROM '/Users/minghuang/Documents/git/Galvanize/lecture-prep/sql-python/ming_huang/data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        '''
cur.execute(query)

#### Step 5: Run a query to get 30 records from our data

In [ ]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''
cur.execute(query)

#### Step 6: Lets take a look at one line of data

In [ ]:
cur.fetchone()

#### Step 7: Lets take a look at 10 lines instead

In [ ]:
cur.fetchmany(10)

#### Step 8: Lets take a look at everything

In [ ]:
cur.fetchall()

If you count the lines, you can observe that the previously fetched line is no longer fetched when you call fetch commands again.  This is similar to generators, where you can only traverse through the output results once.

# Dynamic Queries

#### Example

We have 8 login csv files that we need to insert into the logins table.  Instead of doing a COPY FROM query 8 times, we should utilize Python to automate the process.

#### Step 1: First lets get an idea of how many records we start with

In [ ]:
cur.execute('SELECT count(*) FROM logins;')
cur.fetchall()

#### Step 2: I want to navigate through my file directory, so we need to add os into my namespace

In [ ]:
import os

#### Step 3: Create a query template and determine file path for imports

In [ ]:
query = '''
        COPY logins 
        FROM '{file_path}'
        DELIMITER ','
        CSV;
        '''

folder_path = '/Users/minghuang/Documents/git/Galvanize/lecture-prep/sql-python/ming_huang/data/'

#### Step 4: Use string formatting to generate a query for each approved file.

In [ ]:
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and file_name != 'logins01.csv':
        dyn_query = query.format(file_path = folder_path + file_name)
        cur.execute(dyn_query)
        print '{0} inserted into table.'.format(file_name)

#### Step 5: Lets check how many records we have right now.

In [ ]:
cur.execute('SELECT count(*) FROM logins;')
cur.fetchall()

#### Step 6: Don't forget to commit your changes!

In [ ]:
conn.commit()

#### Step 7: Close your connection

In [ ]:
conn.close()

# Exercise

You're given a file called playgolf.csv in the data folder.  The file is tab delimited and the first row is the header.  Without opening and looking at the file, create a table and insert the data.

In [ ]:
conn.close()